In [109]:
class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False
  
    def preprocessing(self, sequences):
        result = []
        sequences=str(sequences) 
        sequences=sequences.lower()
        sequences=sequences.split(',') 
        for i in sequences:
            i=i.replace('.','').replace('!','').replace(" '",'').replace("'",'').replace('[','').replace(']','').replace('?','') 
            i=i.split(' ') 
            result.append(i)
        return result
  
    def fit(self, sequences):
        self.fit_checker = False
        tokens = self.preprocessing(sequences)
        a=1
        for i in tokens:
            for j in i:
                if j not in self.word_dict.keys():
                    self.word_dict[j]=a
                    a+=1
        self.fit_checker = True
    def transform(self, sequences):
        result = []
        tokens= self.preprocessing(sequences)
        if self.fit_checker: 
            for item in tokens: 
                for i,j in enumerate(item): 
                    if j in self.word_dict.keys():
                        item[i]=self.word_dict[j]
                    if j not in self.word_dict.keys(): 
                        item[i]=self.word_dict['oov']
                result.append(item) 
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result


    

In [110]:
a=Tokenizer()
sen=['I want to go to school','I like Pizza!','I go to school.']
a.fit(sen)
a.transform(sen)

[[1, 2, 3, 4, 3, 5], [1, 6, 7], [1, 4, 3, 5]]

In [111]:
class TfidfVectorizer:
    def __init__(self,tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        import math
        to=[]
        for i in tokenized:
            to.extend(i)
        to=dict.fromkeys(to)
        for a in to:
            num=0
            for j in tokenized:
                if a in j:
                    num+=1
            b=math.log(len(tokenized)/(1+num))
            to[a]=b
        self.to=to  
        self.fit_checker = True
    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)   
            to=self.to               
            self.tfidf_matrix=[]
            for i in tokenized:
                tf=[]
                for j in to:
                    a=i.count(j)*to[j]
                    tf.append(a)
                self.tfidf_matrix.append(tf)
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [112]:
import pandas as pd
b=TfidfVectorizer(a)
sen=['I want to go to school','I like Pizza!','I go to school.']
b.fit(sen)
b.transform(sen)
pd.DataFrame(b.transform(sen),columns=list(b.tokenizer.word_dict.keys())[1:])

,i,want,to,go,school,like,pizza
0,-0.287682,0.405465,0.0,0.0,0.0,0.000000,0.000000
1,-0.287682,0.000000,0.0,0.0,0.0,0.405465,0.405465
2,-0.287682,0.000000,0.0,0.0,0.0,0.000000,0.000000
